# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [5]:
import requests
from bs4 import BeautifulSoup

## Задание 1

In [8]:
URL = 'https://habr.com/ru/search/'
KEYWORDS = ['python', 'парсинг']

params = {
    'q': KEYWORDS,
    'target_type' : 'posts',
    'order_by' : 'relevance'
}

In [9]:
res = requests.get(URL, params)

In [10]:
soup = BeautifulSoup(res.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Результаты поиска по запросу «парсинг» / Хабр</title>
<meta content="71593b225aeafc4e" name="yandex-verification">
<meta content="unsafe-url" name="referrer"/>
<meta content="ed24b2b9721edf0a282c5b4a3232c4" name="pocket-site-verification"/>
<meta content="https://dr.habracdn.net/habr/5f77013e/images/" name="biu"/>
<style type="text/css">
  @font-face{font-family:'Fira Sans';font-style:normal;font-weight:500;src:url(https://dr.habracdn.net/habr/5f77013e/fonts/FiraSans/firaSans-medium.eot);src:local("Fira Sans Medium"),local("FiraSans-Medium"),url(https://dr.habracdn.net/habr/5f77013e/fonts/FiraSans/firaSans-medium.eot?#iefix) format("embedded-opentype"),url(https://dr.habracdn.net/habr/5f77013e/fonts/FiraSans/firaSans-medium.woff2) format("woff2"),url(https://dr.habracdn.net/habr/5f77013e/fonts/FiraSans/firaSans-m